## 코드 검색

[오픈AI 파이썬 코드 저장소](https://github.com/openai/openai-python)의 색인을 생성하고, 이를 검색할 수 있는 방법을 소개합니다. 파이썬 파일에서 함수를 파싱하고 추출하는 간단한 버전을 구현합니다.

We index our own [openai-python code repository](https://github.com/openai/openai-python), and show how it can be searched. We implement a simple version of file parsing and extracting of functions from python files.

In [4]:
import os
from glob import glob
import pandas as pd

def get_function_name(code):
    """
    Extract function name from a line beginning with "def "
    """
    assert code.startswith("def ")
    return code[len("def "): code.index("(")]

def get_until_no_space(all_lines, i) -> str:
    """
    Get all lines until a line outside the function definition is found.
    """
    ret = [all_lines[i]]
    for j in range(i + 1, i + 10000):
        if j < len(all_lines):
            if len(all_lines[j]) == 0 or all_lines[j][0] in [" ", "\t", ")"]:
                ret.append(all_lines[j])
            else:
                break
    return "\n".join(ret)

def get_functions(filepath):
    """
    Get all functions in a Python file.
    """
    whole_code = open(filepath).read().replace("\r", "\n")
    all_lines = whole_code.split("\n")
    for i, l in enumerate(all_lines):
        if l.startswith("def "):
            code = get_until_no_space(all_lines, i)
            function_name = get_function_name(code)
            yield {"code": code, "function_name": function_name, "filepath": filepath}


# get user root directory
root_dir = os.path.expanduser("~")
# note: for this code to work, the openai-python repo must be downloaded and placed in your root directory

# path to code repository directory
code_root = root_dir + "/Documents/GIT/openai-python"

print(code_root)

code_files = [y for x in os.walk(code_root) for y in glob(os.path.join(x[0], '*.py'))]
print("Total number of py files:", len(code_files))

if len(code_files) == 0:
    print("Double check that you have downloaded the openai-python repo and set the code_root variable correctly.")

all_funcs = []
for code_file in code_files:
    funcs = list(get_functions(code_file))
    for func in funcs:
        all_funcs.append(func)

print("Total number of functions extracted:", len(all_funcs))

/Users/wlkim/Documents/GIT/openai-python
Total number of py files: 52
Total number of functions extracted: 101


In [5]:
from openai.embeddings_utils import get_embedding

df = pd.DataFrame(all_funcs)
df['code_embedding'] = df['code'].apply(lambda x: get_embedding(x, engine='text-embedding-ada-002'))
df['filepath'] = df['filepath'].apply(lambda x: x.replace(code_root, ""))
df.to_csv("data/code_search_openai-python.csv", index=False)
df.head()

,code,function_name,filepath,code_embedding
0,def _console_log_level():\n if openai.log i...,_console_log_level,/openai/util.py,"[0.033577464520931244, -0.00459148408845067, 0..."
1,"def log_debug(message, **params):\n msg = l...",log_debug,/openai/util.py,"[-0.004034275189042091, 0.004895383026450872, ..."
2,"def log_info(message, **params):\n msg = lo...",log_info,/openai/util.py,"[0.004854559898376465, 0.003291919594630599, -..."
3,"def log_warn(message, **params):\n msg = lo...",log_warn,/openai/util.py,"[0.002535992069169879, -0.010829543694853783, ..."
4,"def logfmt(props):\n def fmt(key, val):\n ...",logfmt,/openai/util.py,"[0.016732551157474518, 0.017367802560329437, 0..."


In [9]:
from openai.embeddings_utils import cosine_similarity

def search_functions(df, code_query, n=3, pprint=True, n_lines=7):
    embedding = get_embedding(code_query, engine='text-embedding-ada-002')
    df['similarities'] = df.code_embedding.apply(lambda x: cosine_similarity(x, embedding))

    print('-'*70)
    print("code_query:" + code_query)
    print('-'*70)
    res = df.sort_values('similarities', ascending=False).head(n)
    if pprint:
        for r in res.iterrows():
            print(r[1].filepath+":"+r[1].function_name + "  score=" + str(round(r[1].similarities, 3)))
            print("\n".join(r[1].code.split("\n")[:n_lines]))
            print('-'*70)
    return res

res = search_functions(df, 'Completions API tests', n=3)

----------------------------------------------------------------------
code_query:Completions API tests
----------------------------------------------------------------------
/openai/tests/test_endpoints.py:test_completions  score=0.826
def test_completions():
    result = openai.Completion.create(prompt="This was a test", n=5, engine="ada")
    assert len(result.choices) == 5


----------------------------------------------------------------------
/openai/tests/test_endpoints.py:test_completions_model  score=0.811
def test_completions_model():
    result = openai.Completion.create(prompt="This was a test", n=5, model="ada")
    assert len(result.choices) == 5
    assert result.model.startswith("ada")


----------------------------------------------------------------------
/openai/tests/test_endpoints.py:test_completions_multiple_prompts  score=0.808
def test_completions_multiple_prompts():
    result = openai.Completion.create(
        prompt=["This was a test", "This was another test

In [10]:
res = search_functions(df, 'fine-tuning input data validation logic', n=3)

----------------------------------------------------------------------
code_query:fine-tuning input data validation logic
----------------------------------------------------------------------
/openai/validators.py:format_inferrer_validator  score=0.751
def format_inferrer_validator(df):
    """
    This validator will infer the likely fine-tuning format of the data, and display it to the user if it is classification.
    It will also suggest to use ada and explain train/validation split benefits.
    """
    ft_type = infer_task_type(df)
    immediate_msg = None
----------------------------------------------------------------------
/openai/validators.py:get_validators  score=0.748
def get_validators():
    return [
        num_examples_validator,
        lambda x: necessary_column_validator(x, "prompt"),
        lambda x: necessary_column_validator(x, "completion"),
        additional_column_validator,
        non_empty_field_validator,
------------------------------------------------

In [11]:
res = search_functions(df, 'find common suffix', n=2, n_lines=10)

----------------------------------------------------------------------
code_query:find common suffix
----------------------------------------------------------------------
/openai/validators.py:get_common_xfix  score=0.793
def get_common_xfix(series, xfix="suffix"):
    """
    Finds the longest common suffix or prefix of all the values in a series
    """
    common_xfix = ""
    while True:
        common_xfixes = (
            series.str[-(len(common_xfix) + 1) :]
            if xfix == "suffix"
            else series.str[: len(common_xfix) + 1]
----------------------------------------------------------------------
/openai/validators.py:common_completion_suffix_validator  score=0.778
def common_completion_suffix_validator(df):
    """
    This validator will suggest to add a common suffix to the completion if one doesn't already exist in case of classification or conditional generation.
    """
    error_msg = None
    immediate_msg = None
    optional_msg = None
    optional_fn = 

In [14]:
res = search_functions(df, 'Command line interface for fine-tuning', n=3, n_lines=10)

----------------------------------------------------------------------
code_query:Command line interface for fine-tuning
----------------------------------------------------------------------
/openai/cli.py:tools_register  score=0.78
def tools_register(parser):
    subparsers = parser.add_subparsers(
        title="Tools", help="Convenience client side tools"
    )

    def help(args):
        parser.print_help()

    parser.set_defaults(func=help)

----------------------------------------------------------------------
/openai/_openai_scripts.py:main  score=0.763
def main():
    parser = argparse.ArgumentParser(description=None)
    parser.add_argument(
        "-v",
        "--verbose",
        action="count",
        dest="verbosity",
        default=0,
        help="Set verbosity.",
    )
----------------------------------------------------------------------
/openai/cli.py:wandb_register  score=0.762
def wandb_register(parser):
    subparsers = parser.add_subparsers(
        title="

In [15]:
res = search_functions(df, '임베딩', n=3, n_lines=10)

----------------------------------------------------------------------
code_query:임베딩
----------------------------------------------------------------------
/openai/cli.py:organization_info  score=0.673
def organization_info(obj):
    organization = getattr(obj, "organization", None)
    if organization is not None:
        return "[organization={}] ".format(organization)
    else:
        return ""


----------------------------------------------------------------------
/openai/cli.py:api_register  score=0.669
def api_register(parser):
    # Engine management
    subparsers = parser.add_subparsers(help="All API subcommands")

    def help(args):
        parser.print_help()

    parser.set_defaults(func=help)

    sub = subparsers.add_parser("engines.list")
----------------------------------------------------------------------
/openai/cli.py:wandb_register  score=0.667
def wandb_register(parser):
    subparsers = parser.add_subparsers(
        title="wandb", help="Logging with Weights 

In [17]:
res = search_functions(df, '파인튜닝을 위한 API 호출 예제', n=3, n_lines=10)

----------------------------------------------------------------------
code_query:파인튜닝을 위한 API 호출 예제
----------------------------------------------------------------------
/openai/tests/test_util.py:test_openai_api_key_path  score=0.688
def test_openai_api_key_path(api_key_file) -> None:
    print("sk-foo", file=api_key_file)
    api_key_file.flush()
    assert util.default_api_key() == "sk-foo"


----------------------------------------------------------------------
/openai/tests/test_url_composition.py:test_engine_search_url_composition_open_ai  score=0.683
def test_engine_search_url_composition_open_ai() -> None:
    engine = Engine(id="test_id", api_type="open_ai")
    assert engine.api_type == "open_ai"
    assert engine.typed_api_type == ApiType.OPEN_AI
    url = engine.instance_url()
    assert url == "/engines/test_id"


----------------------------------------------------------------------
/openai/cli.py:api_register  score=0.683
def api_register(parser):
    # Engine manageme